In [ ]:
import cv2
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
characters = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'

# Define the dimensions of the license plate image
plate_width = 200
plate_height = 100

# Define the font and font size for the character labels
font = cv2.FONT_HERSHEY_SIMPLEX
font_size = 4
font_thickness = 6

# Define the number of images to create
num_images = 1000

# Create the images and labels
images = []
labels = []
for i in range(100):
    # Create a blank white image
    image = np.ones((plate_height, plate_width, 3), dtype=np.uint8) * 255

    # Choose a random character for each of the 7 spots on the license plate
    label = ''
    for j in range(7):
        character = characters[np.random.randint(len(characters))]
        label += character

        # Draw the character on the image
        text_size, _ = cv2.getTextSize(character, font, font_size, font_thickness)
        x = int((plate_width / 7) * (j + 0.5) - text_size[0] / 2)
        y = int(plate_height / 2 + text_size[1] / 2)
        cv2.putText(image, character, (x, y), font, font_size, (0, 0, 0), font_thickness)

    # Add some random noise to the image
    noise = np.random.normal(loc=0, scale=20, size=(plate_height, plate_width, 3)).astype(np.int32)
    image = np.clip(image.astype(np.int32) + noise, 0, 255).astype(np.uint8)

    # Add the image and label to the dataset
    images.append(image)
    labels.append(label)


In [ ]:
def create_plate_graph(image):
    # Convert the image to grayscale
    print(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary image
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Create a graph
    graph = nx.Graph()

    # Add nodes to the graph for each character in the license plate
    for i, contour in enumerate(contours):
        x,y,w,h = cv2.boundingRect(contour)
        graph.add_node(i, pos=(x,y))

    # Add edges to the graph for characters that are close together
    for i in range(len(contours)):
        for j in range(i+1, len(contours)):
            if np.linalg.norm(np.array(graph.nodes[i]['pos']) - np.array(graph.nodes[j]['pos'])) < 50:
                graph.add_edge(i, j)

    return graph


In [ ]:
# Define a GNN model
class PlateGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PlateGNN, self).__init__()
        self.conv1 = nn.Conv2d(input_dim, hidden_dim, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(hidden_dim, output_dim, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(output_dim, output_dim)
        self.fc2 = nn.Linear(output_dim, output_dim)
        self.fc3 = nn.Linear(output_dim, output_dim)

    def forward(self, x, adj):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        x = torch.matmul(adj, x)
        return x

# Define the loss function
def loss_fn(output, labels):
    loss = F.cross_entropy(output, labels)
    return loss



# Create the GNN model
model = PlateGNN(input_dim=1, hidden_dim=16, output_dim=10)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i in range(len(images)):
        # Create a graph representation of the license plate image
        graph = create_plate_graph(images[i])

        # Create an adjacency matrix from the graph
        adj = nx.adjacency_matrix(graph)
        print(adj)
        adj = torch.FloatTensor(adj.toarray())

        # Create a tensor of node features (all 0s)
        node_features = torch.zeros(adj.shape[0], 1)

        # Feed the graph and node features into the model to predict character labels
        output = model(node_features, adj)

        # Calculate the loss
        loss = loss_fn(output, torch.LongTensor([labels[i]]))

        # Zero the gradients, backward propagate, and update the model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update the running loss
        running_loss += loss.item()

    # Print the epoch number and running loss
    print('Epoch [%d], loss: %.4f' % (epoch+1, running_loss / len(images)))


[[[255 255 255]
  [237 255 254]
  [249 255 252]
  ...
  [255 255 255]
  [255 255 247]
  [255 255 255]]

 [[242 255 255]
  [213 238 238]
  [245 240 246]
  ...
  [211 255 229]
  [255 255 255]
  [251 240 255]]

 [[255 255 255]
  [254 255 255]
  [255 228 255]
  ...
  [253 229 255]
  [255 255 255]
  [235 219 255]]

 ...

 [[239 255 255]
  [252 239 239]
  [227 254 255]
  ...
  [226 245 255]
  [255 255 255]
  [255 255 241]]

 [[223 255 252]
  [243 238 255]
  [255 250 244]
  ...
  [243 218 252]
  [255 255 255]
  [254 226 234]]

 [[255 239 255]
  [255 235 254]
  [248 252 248]
  ...
  [221 243 235]
  [246 255 255]
  [230 255 245]]]
  (0, 2)	1
  (0, 3)	1
  (1, 3)	1
  (1, 4)	1
  (2, 0)	1
  (2, 5)	1
  (3, 0)	1
  (3, 1)	1
  (3, 8)	1
  (4, 1)	1
  (4, 6)	1
  (4, 9)	1
  (4, 10)	1
  (5, 2)	1
  (5, 13)	1
  (6, 4)	1
  (6, 7)	1
  (6, 9)	1
  (6, 10)	1
  (7, 6)	1
  (7, 9)	1
  (7, 10)	1
  (8, 3)	1
  (8, 10)	1
  (8, 11)	1
  (8, 12)	1
  (9, 4)	1
  (9, 6)	1
  (9, 7)	1
  (9, 10)	1
  (10, 4)	1
  (10, 6)	1
  (10, 7

RuntimeError: ignored